<a href="https://colab.research.google.com/github/armandossrecife/teste/blob/main/analise_tags_cassandra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Dados da máquina virtual

In [1]:
distro_linux = !lsb_release -a
memoria_ram = !free -h
print(distro_linux)
print(memoria_ram)

['No LSB modules are available.', 'Distributor ID:\tUbuntu', 'Description:\tUbuntu 20.04.5 LTS', 'Release:\t20.04', 'Codename:\tfocal']
['              total        used        free      shared  buff/cache   available', 'Mem:           12Gi       558Mi       8.2Gi       1.0Mi       3.9Gi        11Gi', 'Swap:            0B          0B          0B']


# Funções de apoio

In [38]:
import os
import pandas as pd

def remove_diretorio_existente(directory):
  if os.path.isdir(directory):
      os.environ['DIRECTORY'] = directory
      print(f"The directory {directory} exists.")
      print(f"Deleting {directory}...")
      comando = !rm -rf $DIRECTORY
      print("Finished!")
  else:
      print(f"The directory {directory} does not exist.")

def get_total_arquivos(directory):
  os.environ['DIRECTORY'] = directory
  total_arquivos = !echo "Total de arquivos: " && find $DIRECTORY -type f | wc -l
  total_arquivos = int(total_arquivos[1])
  return total_arquivos

def get_lista_tags(directory):
  os.environ['DIRECTORY'] = directory
  lista_tags = !cd $DIRECTORY && git tag
  return lista_tags

def get_total_arquivos_branch(directory, my_tag):
  os.environ['DIRECTORY'] = directory
  os.environ['MY_TAG'] = my_tag
  comando = !cd $DIRECTORY && git checkout $MY_TAG && echo "Total de arquivos: " && find . -type f | wc -l
  total_arquivos = int(comando[-1])
  return total_arquivos

def get_types_of_files_branch(directory, my_tag):
  os.environ['DIRECTORY'] = directory
  os.environ['MY_TAG'] = my_tag
  comando1 = !cd $DIRECTORY && git checkout $MY_TAG
  comando2 = !cd $DIRECTORY && cloc .
  return comando2

def get_texto_cloc_para_df(directory, my_tag):
  os.environ['DIRECTORY'] = directory
  os.environ['EXTENSION'] = '_cloc.txt'
  os.environ['MY_TAG'] = my_tag
  comando1 = !cd $DIRECTORY && git checkout $MY_TAG
  comando2 = !cd $DIRECTORY && cloc . --out=$DIRECTORY$EXTENSION
  my_cloc = directory + '_cloc.txt'
  my_file = directory + '/' + my_cloc
  with open(my_file, 'r') as f:
      lines = f.readlines()[1:]
  with open(my_file, 'w') as f:
      f.writelines(lines)
  # read the text file into a dataframe
  df = pd.read_table(my_file, sep='\s{2,}', engine='python', skiprows=[0, 15, 16], header=None, names=['Language', 'files', 'blank', 'comment', 'code'], index_col='Language')
  # display the dataframe
  df
  return df

# Prepara ambiente

In [4]:
remove_diretorio_existente(directory='cassandra')
remove_diretorio_existente(directory='tag_300')
!mkdir tag_300
remove_diretorio_existente(directory='tag_31111')
!mkdir tag_31111

The directory cassandra does not exist.
The directory tag_300 does not exist.
The directory tag_31111 does not exist.


# Análise dos arquivos e tipos de linguagem

In [5]:
!git clone https://github.com/apache/cassandra.git

Cloning into 'cassandra'...
remote: Enumerating objects: 384453, done.
remote: Counting objects: 100% (154/154), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 384453 (delta 49), reused 136 (delta 47), pack-reused 384299
Receiving objects: 100% (384453/384453), 385.21 MiB | 15.73 MiB/s, done.
Resolving deltas: 100% (223956/223956), done.


In [6]:
!sudo apt-get install cloc

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libalgorithm-diff-perl libalgorithm-diff-xs-perl libb-hooks-op-check-perl
  libclass-method-modifiers-perl libclass-xsaccessor-perl
  libdevel-callchecker-perl libdynaloader-functions-perl libimport-into-perl
  libmodule-runtime-perl libmoo-perl libparallel-forkmanager-perl
  libparams-classify-perl libregexp-common-perl librole-tiny-perl
  libstrictures-perl libsub-name-perl libsub-quote-perl
Suggested packages:
  libscalar-number-perl libbareword-filehandles-perl libindirect-perl
  libmultidimensional-perl
The following NEW packages will be installed:
  cloc libalgorithm-diff-perl libalgorithm-diff-xs-perl
  libb-hooks-op-check-perl libclass-method-modifiers-perl
  libclass-xsaccessor-perl libdevel-callchecker-perl
  libdynaloader-functions-perl libimport-into-perl libmodule-runtime-perl
  libmoo-perl libparallel-forkmanager-perl libpa

In [7]:
print(f"Total de arquivos: {get_total_arquivos(directory='cassandra')}")

Total de arquivos: 5228


# Lista de tags para análise

lista_tags_cassandra = ['cassandra-3.0.0', 'cassandra-3.1', 'cassandra-3.2', 'cassandra-3.3', 'cassandra-3.4', 'cassandra-3.5', 'cassandra-3.6', 'cassandra-3.7','cassandra-3.8','cassandra-3.9','cassandra-3.10','cassandra-3.11.11']

In [60]:
lista_tags_cassandra = ['cassandra-3.0.0', 'cassandra-3.1', 'cassandra-3.2', 'cassandra-3.3', 'cassandra-3.4', 'cassandra-3.5', 'cassandra-3.6', 'cassandra-3.7','cassandra-3.8','cassandra-3.9','cassandra-3.10','cassandra-3.11.11']

print(f"Qtd de tags: {len(lista_tags_cassandra)}, {lista_tags_cassandra}")

Qtd de tags: 12, ['cassandra-3.0.0', 'cassandra-3.1', 'cassandra-3.2', 'cassandra-3.3', 'cassandra-3.4', 'cassandra-3.5', 'cassandra-3.6', 'cassandra-3.7', 'cassandra-3.8', 'cassandra-3.9', 'cassandra-3.10', 'cassandra-3.11.11']


In [61]:
dict_total_arquivos = {}
dict_dfs = {}
for each_tag in lista_tags_cassandra:
  print(f'Analisando a tag {each_tag}')
  dict_total_arquivos[each_tag] = get_total_arquivos_branch(directory='cassandra', my_tag=each_tag)
  df = get_texto_cloc_para_df(directory='cassandra', my_tag=each_tag)
  dict_dfs[each_tag] = df
  print('Análise concluída!')

Analisando a tag cassandra-3.0.0
Análise concluída!
Analisando a tag cassandra-3.1
Análise concluída!
Analisando a tag cassandra-3.2
Análise concluída!
Analisando a tag cassandra-3.3
Análise concluída!
Analisando a tag cassandra-3.4
Análise concluída!
Analisando a tag cassandra-3.5
Análise concluída!
Analisando a tag cassandra-3.6
Análise concluída!
Analisando a tag cassandra-3.7
Análise concluída!
Analisando a tag cassandra-3.8
Análise concluída!
Analisando a tag cassandra-3.9
Análise concluída!
Analisando a tag cassandra-3.10
Análise concluída!
Analisando a tag cassandra-3.11.11
Análise concluída!


In [63]:
print(dict_total_arquivos['cassandra-3.0.0'])
dict_dfs['cassandra-3.0.0']

2409


,files,blank,comment,code
Language,,,,
Language,files,blank,comment,code
-------------------------------------------------------------------------------,None,None,None,None
Java,1763,58813,67779,316059
Python,25,1811,3007,5713
Ant,3,174,204,1915
ANTLR Grammar,1,198,221,1473
Bourne Shell,34,293,775,1295
PowerShell,4,127,203,808
DOS Batch,19,178,70,683
